In [54]:
import pandas as pd

transactions = [
    ["A", "B"],
    ["B", "C", "D"],
    ["A", "C", "D", "E"],
    ["A", "D", "E"],
    ["A", "B", "C"],
    ["A", "B", "C", "D"],
    ["B", "C"],
    ["A", "B", "C"],
    ["A", "B", "D"],
    ["B", "C", "E"],
]

df = pd.DataFrame({"items": transactions})
df

,items
0,"[A, B]"
1,"[B, C, D]"
2,"[A, C, D, E]"
3,"[A, D, E]"
4,"[A, B, C]"
5,"[A, B, C, D]"
6,"[B, C]"
7,"[A, B, C]"
8,"[A, B, D]"
9,"[B, C, E]"


In [55]:
df_encoded = df["items"].str.join("|").str.get_dummies().astype(bool)
df_encoded

,A,B,C,D,E
0,True,True,False,False,False
1,False,True,True,True,False
2,True,False,True,True,True
3,True,False,False,True,True
4,True,True,True,False,False
5,True,True,True,True,False
6,False,True,True,False,False
7,True,True,True,False,False
8,True,True,False,True,False
9,False,True,True,False,True


In [56]:
from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets = fpgrowth(df_encoded, min_support=0.2, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.8,(B)
1,0.7,(A)
2,0.7,(C)
3,0.5,(D)
4,0.3,(E)
5,0.5,"(A, B)"
6,0.4,"(A, C)"
7,0.3,"(A, C, B)"
8,0.6,"(C, B)"
9,0.3,"(D, C)"


In [58]:
from mlxtend.frequent_patterns import association_rules

certain_rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=1.0,
    num_itemsets=len(frequent_itemsets),
)

certain_rules[["antecedents", "consequents", "support", "confidence"]]

,antecedents,consequents,support,confidence
0,"(D, E)",(A),0.2,1.0
1,"(E, A)",(D),0.2,1.0


In [59]:
def filter_rules_format(rules, num_antecedents, num_consequents):
    return rules[
        (rules["antecedents"].apply(len) == num_antecedents)
        & (rules["consequents"].apply(len) == num_consequents)
    ]

rules_2_1 = filter_rules_format(certain_rules, 2, 1)
rules_2_1[["antecedents", "consequents", "support", "confidence"]]

,antecedents,consequents,support,confidence
0,"(D, E)",(A),0.2,1.0
1,"(E, A)",(D),0.2,1.0
